In [3]:
import pandas as pd
import re

try:
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    df = tables[0]

    column_map = {
        'Symbol': 'ticker',
        'Security': 'name',
        'Date added': 'date_added',
        'Date first added': 'date_added'
    }

    available_columns = [col for col in column_map if col in df.columns]
    if not available_columns:
        raise ValueError("No expected columns found in the table")

    df = df[available_columns].rename(columns=column_map)

    def extract_year(date):
        if pd.isna(date):
            return None
        match = re.search(r'\d{4}', str(date))
        return int(match.group()) if match else None

    if 'date_added' in df.columns:
        df['year_added'] = df['date_added'].apply(extract_year)
    else:
        df['year_added'] = None

    yearly_counts = df['year_added'].value_counts().sort_index()
    yearly_counts = yearly_counts[yearly_counts.index != 1957]

    if yearly_counts.empty:
        raise ValueError("No valid years found after processing")

    max_count = yearly_counts.max()
    max_years = yearly_counts[yearly_counts == max_count].index
    most_recent_max_year = int(max_years.max())

    print(f"Year with the highest number of additions: {most_recent_max_year}")

    df.to_csv('sp500_companies.csv', index=False)
    yearly_counts.to_csv('sp500_yearly_additions.csv', header=['count'])

except Exception as e:
    print(f"Error occurred: {str(e)}")

Year with the highest number of additions: 2017


In [ ]:
!pip install yfinance pandas

In [16]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime

indices = {
    'S&P 500': '^GSPC',
    'Shanghai Composite': '000001.SS',
    'Hang Seng': '^HSI',
    'S&P/ASX 200': '^AXJO',
    'Nifty 50': '^NSEI',
    'S&P/TSX Composite': '^GSPTSE',
    'DAX': '^GDAXI',
    'FTSE 100': '^FTSE',
    'Nikkei 225': '^N225',
    'IPC Mexico': '^MXX',
    'Ibovespa': '^BVSP'
}

start_date = '2025-01-01'
end_date = '2025-04-30'

returns = {}
for name, ticker in indices.items():
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
    if not data.empty and 'Close' in data.columns:
        close_data = data['Close'].dropna()
        if len(close_data) >= 2:
            start_price = close_data.iloc[0].item()
            end_price = close_data.iloc[-1].item()
            if isinstance(start_price, (int, float)) and isinstance(end_price, (int, float)):
                ytd_return = ((end_price - start_price) / start_price) * 100
                returns[name] = ytd_return
    time.sleep(1)

df = pd.DataFrame.from_dict(returns, orient='index', columns=['YTD Return (%)'])
df = df.sort_values(by='YTD Return (%)', ascending=False)

sp500_return = df.loc['S&P 500', 'YTD Return (%)']
better_than_sp500 = len(df[df['YTD Return (%)'] > sp500_return])

print(f"YTD Returns as of {end_date}:")
print(df)
print(f"\nNumber of indexes with better YTD returns than S&P 500: {better_than_sp500}")

YTD Returns as of 2025-04-30:
                    YTD Return (%)
Ibovespa                 12.460354
Hang Seng                12.152832
DAX                      11.991065
IPC Mexico               11.751647
Nifty 50                  2.497794
FTSE 100                  2.462445
Shanghai Composite        0.738499
S&P/TSX Composite        -0.094385
S&P/ASX 200              -1.592451
S&P 500                  -5.243539
Nikkei 225               -8.820460

Number of indexes with better YTD returns than S&P 500: 9


In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

start_date = '1950-01-01'
end_date = '2025-05-31'
data = yf.download('^GSPC', start=start_date, end=end_date, auto_adjust=True, progress=False)

prices = data['Close'].squeeze().dropna()

all_time_highs = []
max_price = -float('inf')
for date, price in prices.items():
    if isinstance(price, (int, float)) and not pd.isna(price):
        if price > max_price:
            all_time_highs.append((date, price))
            max_price = price

corrections = []
for i in range(len(all_time_highs) - 1):
    start_date, high_price = all_time_highs[i]
    end_date, next_high = all_time_highs[i + 1]
    
    period_data = prices[(prices.index >= start_date) & (prices.index < end_date)]
    if not period_data.empty:
        min_price = period_data.min()
        min_date = period_data.idxmin()
        
        if not pd.isna(min_price) and not pd.isna(min_date):
            drawdown = (high_price - min_price) / high_price * 100
            if drawdown >= 5:
                duration = (min_date - start_date).days
                if duration > 0:
                    corrections.append({
                        'start_date': start_date,
                        'min_date': min_date,
                        'high_price': high_price,
                        'min_price': min_price,
                        'drawdown': drawdown,
                        'duration': duration
                    })

corrections_df = pd.DataFrame(corrections)
median_duration = np.percentile(corrections_df['duration'], 50)
print(f"50th (median): {median_duration:.0f}")

50th (median): 39


In [58]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

# Load earnings data
csv_file = r"C:\Users\GabrielF\OneDrive - Freddy Hirsch Group (Pty) Ltd\Desktop\ha1_Amazon.csv"
earnings_df = pd.read_csv(csv_file, delimiter=';')

# Parse Earnings Date
earnings_df['Date'] = pd.to_datetime(earnings_df['Earnings Date'].str.split(' at ').str[0], errors='coerce')

# Adjust for after-hours announcements (after 4 PM)
earnings_df['Time'] = earnings_df['Earnings Date'].str.extract(r'at (\d+:\d+ [AP]M [A-Z]+)')
earnings_df['Date'] = earnings_df.apply(
    lambda row: row['Date'] + timedelta(days=1) if 'PM' in str(row['Time']) and int(row['Time'].split(':')[0]) >= 4 else row['Date'],
    axis=1
)

# Convert Surprise (%) to numeric
earnings_df['Surprise (%)'] = pd.to_numeric(earnings_df['Surprise (%)'], errors='coerce')

# Filter out future dates and invalid dates
earnings_df = earnings_df[earnings_df['Date'] <= '2025-05-31'].dropna(subset=['Date'])

# Identify positive earnings surprises (Surprise > 5%)
positive_surprises = earnings_df[earnings_df['Surprise (%)'] > 5].copy()

# Download AMZN price data
ticker = 'AMZN'
start_date = earnings_df['Date'].min() - timedelta(days=365)
end_date = earnings_df['Date'].max() + timedelta(days=365)
price_data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
prices = price_data['Close'].squeeze()

# Calculate 2-day returns
returns = []
return_dates = []
for i in range(2, len(prices)):
    date1 = prices.index[i - 2]  # Day before reaction (Day 1)
    date3 = prices.index[i]      # Two days after reaction (Day 3)
    close1 = prices.iloc[i - 2]
    close3 = prices.iloc[i]
    if pd.isna(close1) or pd.isna(close3):
        continue
    two_day_return = (close3 / close1) - 1
    returns.append(two_day_return * 100.0)
    return_dates.append(prices.index[i - 1])  # Reaction day (Day 2)

returns_df = pd.DataFrame({'Date': return_dates, '2_Day_Return': returns})
returns_df['Date'] = pd.to_datetime(returns_df['Date'])

# Merge with positive surprises
merged_df = positive_surprises.merge(
    returns_df,
    how='left',
    left_on='Date',
    right_on='Date'
)
merged_df = merged_df.dropna(subset=['2_Day_Return'])

# Calculate median
median_positive = np.median(merged_df['2_Day_Return'])

# Output
print(f"{median_positive:.1f}")

2.7
